In [1]:
!pip install -qq datasets transformers[sentencepiece]
!pip install rouge_score -qq

In [1]:
import glob
import concurrent.futures
import pandas as pd
from transformers import pipeline
import numpy as np
import evaluate  

In [4]:
!wget 'https://github.com/ThanhChinhBK/vietnews/archive/master.zip'
!unzip 'master.zip'

'wget' is not recognized as an internal or external command,
operable program or batch file.
'unzip' is not recognized as an internal or external command,
operable program or batch file.


In [2]:
def listPaths(path):
  pathfiles = list()
  for pathfile in glob.glob(path):
    pathfiles.append(pathfile)
  return pathfiles

test_paths = listPaths('D:/vietnews/data/script/*')

def read_content(pathfile):
    """
    Input: Path of txt file
    Output: A dictionary has keys 'original' and 'summary'
    """
    try:
        with open(pathfile, encoding="utf-8") as f:  # Thêm encoding UTF-8
            rows = f.readlines()
            if len(rows) < 5:  # Kiểm tra file có đủ dòng không
                return None
            original = ' '.join(''.join(rows[4:]).split('\n'))
            summary = ' '.join(rows[2].split('\n'))

        return {'file': pathfile, 'original': original, 'summary': summary}
    except UnicodeDecodeError:
        print(f"Encoding error in file: {pathfile}")
        return None  # Bỏ qua file bị lỗi



def get_dataframe(pathfiles):
    with concurrent.futures.ThreadPoolExecutor() as executor:  # Dùng ThreadPoolExecutor thay vì ProcessPoolExecutor
        data = list(executor.map(read_content, pathfiles))

    # Loại bỏ các giá trị None (các tệp bị lỗi)
    data_df = [d for d in data if d is not None]
    data_df = pd.DataFrame(data_df)
    data_df.dropna(inplace=True)
    data_df = data_df.sample(frac=1).reset_index(drop=True)

    return data_df   

In [3]:
test_df = get_dataframe(test_paths)

In [4]:
test_df.head(3)

,file,original,summary
0,D:/vietnews/data/script\10.txt,"Ê nó nè, Tết rồi trả tiền tôi được không? Biế...","Đâu có vỗ, cuộc đời vỗ với mặt em đó Bây giờ ..."
1,D:/vietnews/data/script\6.txt,"em về, tao lo cho bản mặt già của mày rồi tao...",Mày mới ăn đục thừa đầu cặn của tao nữa con A...
2,D:/vietnews/data/script\2.txt,"được giấy đăng ký kết hôn, chủ tịch đã thú nh...",Chủ tịch vừa đến cửa sở dân chính thì cô gái ...


In [5]:
test_df['original'][0]

' Ê nó nè, Tết rồi trả tiền tôi được không? Biết sao không? Tết rồi nhiều khoản phải lo quá à Thôi tôi lấy trước vô đây nha Cuối năm nay, công ty chúng ta sẽ hợp tác với lại nhãn hàng Lai Boy Vì vậy cho nên những nhân viên chính thức của công ty sẽ nhận được một phần quà linh đình Mà chưa hết, mọi người ngồi ra  Còn được bóc thăm bao lì xì may mắn nữa Đó Ai may mắn thì sẽ bóc và trúng tiền thưởng Tết đó nha Cô mới làm lao công được 2 tuần Chưa là nhân viên chính thức Nên Cô không được nhận phần quà này Hình như là nhà cô cũng bị ảnh hưởng từ bão lũ năm nay đúng không? Con tặng cô Con đúng là một chàng trai tốt bụng Cảm ơn con nhiều lắm Cảm ơn con nhiều lắm  Lai ơi Lai Ồ chị xin lỗi em nha Cái bản dự chi ban đầu á Chị nhìn lộn tên em hành tên người khác Cho nên em làm đúng rồi Chút nữa chị vô chị ký cái tạm ứng cho em Em không cần phải lấy tiền túi nữa Tết này có tiền ăn Tết rồi Thật hả chị Thôi khoan đi Ủa đúng tên em rồi nè Vậy là chị đúng chứ chị đâu có sai Rồi giống như ban đầu luôn

In [6]:
rouge_score = evaluate.load("rouge")

In [7]:
model_name = "creamyngreen/vietnamese-summarization"

In [8]:
from transformers import MBartForConditionalGeneration
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = MBartForConditionalGeneration.from_pretrained(model_name)

def old_summarization_pipeline(df):
    input_ids = tokenizer.encode_plus(df['original'], return_tensors='pt', max_length=512, truncation=True, padding=True)['input_ids']
    summary_ids = model.generate(input_ids, max_length=128)
    summaries = [tokenizer.decode(s, skip_special_tokens=True) for s in summary_ids]
    return summaries

In [12]:
test_df_sampled = test_df.sample(len(test_df))

In [13]:
predictions = [old_summarization_pipeline(test_df.iloc[i])[0] for i in range(len(test_df))]

In [14]:
references = list(test_df['summary']) 

In [15]:
rouge_score.compute(predictions=predictions,references=references)

{'rouge1': np.float64(0.38749293247111194),
 'rouge2': np.float64(0.08527578540980722),
 'rougeL': np.float64(0.23560593102510627),
 'rougeLsum': np.float64(0.23574571774846068)}

In [16]:
rouge1_fmeasure = rouge_score.compute(predictions=predictions, references=references)['rouge1']
print(rouge1_fmeasure)


0.38749293247111194


In [17]:
rouge2_fmeasure = rouge_score.compute(predictions=predictions, references=references)['rouge2']
print(rouge2_fmeasure)

0.08527578540980722


In [18]:
rougeL_fmeasure = rouge_score.compute(predictions=predictions, references=references)['rougeL']
print(rougeL_fmeasure)

0.23560593102510627


# RESULT DATAFRAME

In [19]:
result_df = pd.DataFrame({'summary':references, 'predictions':predictions})    

In [31]:
print(result_df.iloc[3]['summary'])

 Nói sao nghe tết quớt kìa trời, đẹp thì có đẹp á, mà sao nắng dách vậy? Ở đây vui quán cà phê, chứ không phải văn bún bò. À, vậy hả chị? Nãy giờ em đeo kính, em không có biết. Chị cho em ly coffee nha chị. Coffee là gì? Cà phê đó chị. Cà phê? Vâng. Yes. Đây là cà phê coffee, đợi đợi đi xíu nha. Ok. 


In [30]:
print(result_df.iloc[3]['predictions'])

Em Chế Công - nhân_viên của quán cà_phê mà tôi ghé thăm đã nói với tôi như_vậy .


In [35]:
print(result_df.head())  # Check if 'summary' and 'predictions' exist


                                             summary  \
0   Đâu có vỗ, cuộc đời vỗ với mặt em đó Bây giờ ...   
1   Mày mới ăn đục thừa đầu cặn của tao nữa con A...   
2   Chủ tịch vừa đến cửa sở dân chính thì cô gái ...   
3   Nói sao nghe tết quớt kìa trời, đẹp thì có đẹ...   
4   Hãy subscribe cho kênh Ghiền Mì Gõ Để không b...   

                                         predictions  
0  Cô làm lao công được 2 tuần , chưa có nhân viê...  
1  Nhân Bạch Nhân , Công nhận không đồ hàng ăn ng...  
2  Một người phụ_nữ ở thành_phố Thượng_Hải , Trun...  
3  Em Chế Công - nhân_viên của quán cà_phê mà tôi...  
4  In Ki-ung , cháu gái của In Ki-ung , người đã ...  


In [26]:
result_df.to_csv("movie_test_summarization")